<a href="https://colab.research.google.com/github/dhruv5903/summerinternship24/blob/main/TestRAG_pipeline(Draft1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Test RAG pipeline**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install langchain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 975.5/975.5 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.8/332.8 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.4/127.4 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 145.0/145.0 kB 10.2 MB/s eta 0:00:00


In [ ]:
!pip install -q sentence-transformers
!pip install -q wikipedia-api
!pip install -q numpy
!pip install -q scipy

## **Data Acquisation and Chunking**

In [ ]:
from wikipediaapi import Wikipedia
wiki = Wikipedia('RAGBot/0.0', 'en')
doc = wiki.page('COVID-19 pandemic').text
paragraphs = doc.split('\n\n')

In [ ]:
import textwrap
for i, p in enumerate(paragraphs):
  wrapped_text = textwrap.fill(p, width=100)
  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")

-----------------------------------------------------------------
The COVID-19 pandemic (also known as the coronavirus pandemic), caused by severe acute respiratory
syndrome coronavirus 2 (SARS-CoV-2), began with an outbreak in Wuhan, China, in December 2019. It
spread to other areas of Asia, and then worldwide in early 2020. The World Health Organization (WHO)
declared the outbreak a public health emergency of international concern (PHEIC) on 30 January 2020,
and assessed the outbreak had become a pandemic on 11 March. COVID-19 symptoms range from
asymptomatic to deadly, but most commonly include fever, sore throat, nocturnal cough, and fatigue.
Transmission of the virus is often through airborne particles. Mutations have produced many strains
(variants) with varying degrees of infectivity and virulence. COVID-19 vaccines were developed
rapidly and deployed to the general public beginning in December 2020, made available through
government and international programs such as COVAX, aim

## **Embedding Model**

In [ ]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer("Alibaba-NLP/gte-base-en-v1.5", trust_remote_code=True)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when p

## **Embedding data by the model**

In [ ]:
docs_embed = model.encode(paragraphs, normalize_embeddings=True)

In [ ]:
docs_embed.shape

(81, 768)

In [ ]:
docs_embed[0]

array([-2.04445459e-02, -1.38573535e-02,  3.94676402e-02, -3.17685306e-02,
        1.82236061e-02, -4.47230861e-02,  6.01554438e-02,  8.88456702e-02,
        2.58636698e-02, -2.92455871e-02, -2.61343103e-02,  1.94874238e-02,
       -9.46983397e-02, -1.34090688e-02,  1.35008032e-02, -3.22499909e-02,
        6.19685687e-02,  1.17813103e-01,  7.51708681e-03, -4.09314968e-02,
       -3.10704854e-05, -4.51094508e-02, -6.63978793e-03, -1.21147055e-02,
       -1.06476517e-02,  5.37724234e-03,  7.30383769e-02,  4.03507799e-02,
       -2.83361841e-02, -1.56745650e-02, -6.74934313e-02, -5.17772511e-03,
       -3.95469144e-02, -1.14775607e-02, -2.14089658e-02, -7.17463577e-03,
        1.25020873e-02,  3.88217792e-02, -1.49983894e-02, -1.15851760e-02,
       -5.42138629e-02, -5.06187649e-03, -4.06807847e-02,  4.65075597e-02,
       -1.19288582e-02,  1.12020578e-02, -3.14353332e-02,  3.59876044e-02,
       -3.63145731e-02, -5.02470545e-02,  1.78467575e-02, -1.25404978e-02,
        2.49395669e-02, -

## **Embedding query by the model**

In [ ]:
query = "Symptoms of Coronavirus"
query_embed = model.encode(query, normalize_embeddings=True)

In [ ]:
query_embed.shape

(768,)

##**Closest Paragraph to Query**

In [ ]:
import numpy as np
similarities = np.dot(docs_embed, query_embed.T)
similarities.shape

(81,)

In [ ]:
similarities

array([0.43935952, 0.25833625, 0.35193366, 0.27826416, 0.34714237,
       0.28979474, 0.19712734, 0.17566222, 0.1837004 , 0.6728056 ,
       0.32011855, 0.35525924, 0.25719243, 0.36506435, 0.24931157,
       0.38855195, 0.34626654, 0.18100157, 0.20691442, 0.12608843,
       0.16344789, 0.17366454, 0.10722291, 0.15730307, 0.15434867,
       0.2190742 , 0.28687423, 0.42327031, 0.2492544 , 0.20913126,
       0.2377915 , 0.20904084, 0.22919087, 0.27397037, 0.28729284,
       0.14133528, 0.3443834 , 0.39581624, 0.27075517, 0.31705558,
       0.30012304, 0.33742535, 0.2798286 , 0.22924858, 0.19583023,
       0.2995103 , 0.2346819 , 0.23607758, 0.23710224, 0.19776341,
       0.18798132, 0.21956593, 0.19942936, 0.18552954, 0.18852928,
       0.21765813, 0.2862767 , 0.18975633, 0.30212495, 0.28348827,
       0.1882496 , 0.2684322 , 0.2663741 , 0.28011283, 0.36716974,
       0.20374034, 0.20027855, 0.22084528, 0.24162783, 0.24710533,
       0.24801542, 0.21519865, 0.32883713, 0.22763346, 0.10344

## **Selecting top 3 results**

In [ ]:
top_3_idx = np.argsort(similarities, axis=0)[-3:][::-1].tolist()

In [ ]:
top_3_idx

[9, 0, 27]

In [ ]:
most_similar_documents = [paragraphs[idx] for idx in top_3_idx]

In [ ]:
CONTEXT = ""
for i, p in enumerate(most_similar_documents):
  wrapped_text = textwrap.fill(p, width=100)

  print("-----------------------------------------------------------------")
  print(wrapped_text)
  print("-----------------------------------------------------------------")
  CONTEXT += wrapped_text + "\n\n"

-----------------------------------------------------------------
Signs and symptoms Symptoms of COVID-19 are variable, ranging from mild symptoms to severe illness.
Common symptoms include headache, loss of smell and taste, nasal congestion and runny nose, cough,
muscle pain, sore throat, fever, diarrhoea, and breathing difficulties. People with the same
infection may have different symptoms, and their symptoms may change over time. Three common
clusters of symptoms have been identified: one respiratory symptom cluster with cough, sputum,
shortness of breath, and fever; a musculoskeletal symptom cluster with muscle and joint pain,
headache, and fatigue; a cluster of digestive symptoms with abdominal pain, vomiting, and diarrhoea.
In people without prior ear, nose, and throat disorders, loss of taste combined with loss of smell
is associated with COVID-19 and is reported in as many as 88% of cases.
-----------------------------------------------------------------
----------------------

##**Generation**

In [ ]:
prompt = f"""
use the following CONTEXT to answer the QUESTION at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

CONTEXT: {CONTEXT}
QUESTION: {query}

"""

In [ ]:
!pip install -q openai

In [ ]:
# prompt: write python code to make calls to openai api
from google.colab import userdata
userdata.get('openai')

import openai

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=userdata.get('openai'))

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "user", "content": prompt},
  ]
)

In [ ]:
print(response.choices[0].message.content)